# Anisotropic models

In this tutorial, we demonstrate how to perform estimation with anisotropic variograms using GeoStats.jl.

Before we proceed, please install the following packages:

In [1]:
for pkg in ["GeoStats", "Plots", "GR"]
    Pkg.add(pkg)
end

# make sure this tutorial is reproducible
srand(2000);

INFO: Package GeoStats is already installed
INFO: Package Plots is already installed
INFO: Package GR is already installed


## Ellipsoid distance

Anisotropy can be thought of as a deformation of space with an ellipsoid distance. The semiaxes of the ellipsoid determine the preferential directions of the field and their lengths characterize the anisotropy ratio. In GeoStats.jl, all variogram models (empirical and theoretical) support a custom distance function that
can be used to model anisotropy.

A variogram object $\gamma$ can be evaluated as an isotropic model $\gamma(h)$ or as a (possibly) anisotropic model $\gamma(x,y)$. For the Euclidean distance (the default), these two operations match $\gamma(x,y) = \gamma(h)$ in all directions:

In [2]:
using GeoStats

γ = GaussianVariogram()

γ([1.,0.], [0.,0.]) ≈ γ(1.)

true

If instead of an Euclidean ball, we use an ellipsoid with different semiaxes, the operation $\gamma(x,y)$ becomes a function of the direction $x-y$. For example, we can create an ellipsoid distance aligned with the coordinate system where the major semiaxis has twice the size of the minor semiaxis:

In [3]:
d = EllipsoidDistance([2.,1.],[0.])

γaniso = GaussianVariogram(distance=d)

γaniso([1.,0.],[0.,0.]) ≠ γaniso([0.,1.],[0.,0.])

true

## Effects on estimation

Now that we know how to construct anisotropic variograms, we can investigate the effect of varying the anisotropy ratio and alignement angle on estimation results. We start by generating some random data:

In [4]:
using Plots; gr()
using Interact

dim, nobs = 2, 50

X = 100rand(dim, nobs)
z = rand(nobs)

scatter(X[1,:], X[2,:], c=z, label="data")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data

First, we vary the anisotropy ratio with an ellipsoid that is aligned with the coordinate system:

In [5]:
anim = @animate for a=linspace(1,10,10)
    d = EllipsoidDistance([a,1.], [0.])
    γ = GaussianVariogram(range=5., distance=d)
    
    ordkrig = OrdinaryKriging(X, z, γ)
    
    μ = zeros(100,100)
    σ² = zeros(100,100)
    for i=1:100, j=1:100
        μ[i,j], σ²[i,j] = estimate(ordkrig, Float64[i,j])
    end
    
    plt1 = heatmap(μ', title="Kriging mean")
    plt2 = heatmap(σ²', title="Kriging variance")
    
    plot(plt1, plt2, size=(1000,400))
end
gif(anim, "figs/anisotropy_ratio.gif", fps=1)

INFO: Saved animation to /home/juliohm/.julia/v0.6/GeoStats/examples/figs/anisotropy_ratio.gif


Plots.AnimatedGif("/home/juliohm/.julia/v0.6/GeoStats/examples/figs/anisotropy_ratio.gif")

Second, we fix the anisotropy ratio and vary the alignment angle:

In [6]:
anim = @animate for θ=linspace(0,2π,10)
    d = EllipsoidDistance([5.,1.], [θ])
    γ = GaussianVariogram(range=5., distance=d)
    
    ordkrig = OrdinaryKriging(X, z, γ)
    
    μ = zeros(100,100)
    σ² = zeros(100,100)
    for i=1:100, j=1:100
        μ[i,j], σ²[i,j] = estimate(ordkrig, Float64[i,j])
    end
    
    plt1 = heatmap(μ', title="Kriging mean")
    plt2 = heatmap(σ²', title="Kriging variance")
    
    plot(plt1, plt2, size=(1000,400))
end
gif(anim, "figs/anisotropy_angle.gif", fps=1)

INFO: Saved animation to /home/juliohm/.julia/v0.6/GeoStats/examples/figs/anisotropy_angle.gif


Plots.AnimatedGif("/home/juliohm/.julia/v0.6/GeoStats/examples/figs/anisotropy_angle.gif")

This experiment can be extended to 3D with the only difference being that ellipsoids therein are defined by 3 semiaxes and 3 angles. For example, the Euclidean distance in 3D can be recovered with a degenerated ellipsoid with equal semiaxes (i.e. sphere):

In [7]:
ellipsoid = EllipsoidDistance([1.,1.,1.],[0.,0.,0.])
euclidean = EuclideanDistance()

a, b = rand(3), rand(3)

ellipsoid(a,b) ≈ euclidean(a,b)

true

Please consult the documentation for more distance functions. If a particular distance is not implemented yet, please submit a feature request. Code contributions are also very welcome. One of our goals with GeoStats.jl is to make it easier for users to extend the package.